In [ ]:
import pandas as pd
import numpy as np
import psycopg2

# Load the CSV file into a DataFrame
df = pd.read_csv('your_file.csv')

# Function to create dimension tables
def create_dimension_table(df, key_column, columns, table_name):
    dimension_table = df[columns].drop_duplicates().reset_index(drop=True)
    dimension_table.insert(0, key_column, np.arange(1, len(dimension_table) + 1))

    # Connect to the database
    conn = psycopg2.connect(
        host='your_host',
        port='your_port',
        user='your_user',
        password='your_password',
        database='your_database'
    )

    # Create a database cursor
    cursor = conn.cursor()

    # Create the dimension table in the database
    cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({key_column} INT PRIMARY KEY, {', '.join(columns)});")

    # Insert data into the dimension table
    for index, row in dimension_table.iterrows():
        values = ', '.join(map(str, row.values))
        cursor.execute(f"INSERT INTO {table_name} VALUES ({values});")

    # Commit the changes to the database
    conn.commit()

    # Close the database connection
    conn.close()

# Create dimension tables
create_dimension_table(df, 'tweet_created_key', ['tweet_created'], 'TimeDimension')
create_dimension_table(df, 'tweet_location_key', ['tweet_location'], 'LocationDimension')
create_dimension_table(df, 'user_key', ['name', 'user_timezone'], 'UserDimension')
create_dimension_table(df, 'airline_key', ['airline'], 'AirlineDimension')
create_dimension_table(df, 'sentiment_key', ['airline_sentiment', 'negativereason'], 'SentimentDimension')

# Connect to the database for the fact table
conn = psycopg2.connect(
    host='your_host',
    port='your_port',
    user='your_user',
    password='your_password',
    database='your_database'
)

# Create a database cursor
cursor = conn.cursor()

# Create the fact table in the database
cursor.execute("CREATE TABLE IF NOT EXISTS SentimentFact (tweet_id BIGINT PRIMARY KEY, "
               "tweet_created_key INT, tweet_location_key INT, user_key INT, airline_key INT, sentiment_key INT, "
               "airline_sentiment NVARCHAR(255), airline_sentiment_confidence FLOAT, "
               "negativereason NVARCHAR(255), negativereason_confidence FLOAT, retweet_count INT, text NVARCHAR(MAX), "
               "FOREIGN KEY (tweet_created_key) REFERENCES TimeDimension(tweet_created_key), "
               "FOREIGN KEY (tweet_location_key) REFERENCES LocationDimension(tweet_location_key), "
               "FOREIGN KEY (user_key) REFERENCES UserDimension(user_key), "
               "FOREIGN KEY (airline_key) REFERENCES AirlineDimension(airline_key), "
               "FOREIGN KEY (sentiment_key) REFERENCES SentimentDimension(sentiment_key));")

# Insert data into the fact table
for index, row in df.iterrows():
    values = ', '.join(map(str, row.values))
    cursor.execute(f"INSERT INTO SentimentFact VALUES ({values});")

# Commit the changes to the database
conn.commit()

# Close the database connection
conn.close()
